<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Choosing-the-number-of-planes" data-toc-modified-id="Choosing-the-number-of-planes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Choosing the number of planes</a></span></li><li><span><a href="#Getting-the-hash-number-for-a-vector" data-toc-modified-id="Getting-the-hash-number-for-a-vector-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting the hash number for a vector</a></span><ul class="toc-item"><li><span><a href="#Hyperlanes-in-vector-spaces" data-toc-modified-id="Hyperlanes-in-vector-spaces-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hyperlanes in vector spaces</a></span></li><li><span><a href="#Using-Hyperplanes-to-split-the-vector-space" data-toc-modified-id="Using-Hyperplanes-to-split-the-vector-space-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Using Hyperplanes to split the vector space</a></span></li><li><span><a href="#Encoding-hash-buckets" data-toc-modified-id="Encoding-hash-buckets-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Encoding hash buckets</a></span></li><li><span><a href="#Implementing-hash-buckets" data-toc-modified-id="Implementing-hash-buckets-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Implementing hash buckets</a></span></li></ul></li><li><span><a href="#Creating-a-hash-table" data-toc-modified-id="Creating-a-hash-table-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creating a hash table</a></span></li><li><span><a href="#Approximate-K-NN" data-toc-modified-id="Approximate-K-NN-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Approximate K-NN</a></span></li></ul></div>

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize
import pandas as pd

In [ ]:
N_DIMS = 300
N_VECS = 500
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

N_VECS_PER_BUCKET = 20
print(f"We want to split then into buckets with aprox. {N_VECS_PER_BUCKET} vectors.")

In [ ]:
train_data = normalize(np.random.rand(N_VECS, N_DIMS), axis=1, norm='l2')
print(f"The vector matrix takes up {train_data.nbytes/1024/1024} Gb.")

## Choosing the number of planes

* Each plane divides the space into $2$ parts.
* So $n$ planes divide the space into $2^{n}$ hash buckets.
* We want to organize 500 document vectors into buckets so that every bucket has about $~20$ vectors.
* For that we need $\frac{500}{20}=25$ buckets.
* We're interested in $n$, number of planes, so that $2^{n}= 25$. Now, we can calculate $n=\log_{2}25 =  4.64 \approx 5$.

In [ ]:
# The number of planes. We use log2(25) to have ~20 vectors/bucket.
N_PLANES = int(np.floor(np.log2(N_VECS_PER_BUCKET)))
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 5

<a name="3-4"></a>

## Getting the hash number for a vector

For each vector, we need to get a unique number associated to that vector in order to assign it to a "hash bucket".

### Hyperlanes in vector spaces
* In $3$-dimensional vector space, the hyperplane is a regular plane. In $2$ dimensional vector space, the hyperplane is a line.
* Generally, the hyperplane is subspace which has dimension $1$ lower than the original vector space has.
* A hyperplane is uniquely defined by its normal vector.
* Normal vector $n$ of the plane $\pi$ is the vector to which all vectors in the plane $\pi$ are orthogonal (perpendicular in $3$ dimensional case).

### Using Hyperplanes to split the vector space
We can use a hyperplane to split the vector space into $2$ parts.
* All vectors whose dot product with a plane's normal vector is positive are on one side of the plane.
* All vectors whose dot product with the plane's normal vector is negative are on the other side of the plane.

### Encoding hash buckets
* For a vector, we can take its dot product with all the planes, then encode this information to assign the vector to a single hash bucket.
* When the vector is pointing to the opposite side of the hyperplane than normal, encode it by 0.
* Otherwise, if the vector is on the same side as the normal vector, encode it by 1.
* If you calculate the dot product with each plane in the same order for every vector, you've encoded each vector's unique hash ID as a binary number, like [0, 1, 1, ... 0].

<a name="ex-09"></a>

### Implementing hash buckets

We've initialized hash table `hashes` for you. It is list of `N_UNIVERSES` matrices, each describes its own hash table. Each matrix has `N_DIMS` rows and `N_PLANES` columns. Every column of that matrix is a `N_DIMS`-dimensional normal vector for each of `N_PLANES` hyperplanes which are used for creating buckets of the particular hash table.

*Exercise*: Your task is to complete the function `hash_value_of_vector` which places vector `v` in the correct hash bucket.

* First multiply your vector `v`, with a corresponding plane. This will give you a vector of dimension $(1,\text{N_planes})$.
* You will then convert every element in that vector to 0 or 1.
* You create a hash vector by doing the following: if the element is negative, it becomes a 0, otherwise you change it to a 1.
* You then compute the unique number for the vector by iterating over `N_PLANES`
* Then you multiply $2^i$ times the corresponding bit (0 or 1).
* You will then store that sum in the variable `hash_value`.

**Intructions:** Create a hash for the vector in the function below.
Use this formula:

$$ hash = \sum_{i=0}^{N-1} \left( 2^{i} \times h_{i} \right) $$

In [ ]:
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES))
            for _ in range(N_UNIVERSES)]

In [ ]:
def hash_value_of_vector(v, planes):
    """Create a hash for a vector; hash_id says which random hash to use.
    Input:
        - v:  vector of tweet. It's dimension is (1, N_DIMS)
        - planes: matrix of dimension (N_DIMS, N_PLANES) - the set of planes that divide up the region
    Output:
        - res: a number which is used as a hash for your vector

    """
    # for the set of planes,
    # calculate the dot product between the vector and the matrix containing the planes
    # remember that planes has shape (300, 10)
    # The dot product will have the shape (1,10)
    dot_product = np.matmul(v, planes)

    # get the sign of the dot product (1,10) shaped vector
    sign_of_dot_product = np.sign(dot_product)

    # set h to be false (eqivalent to 0 when used in operations) if the sign is negative,
    # and true (equivalent to 1) if the sign is positive (1,10) shaped vector
    # if the sign is 0, i.e. the vector is in the plane, consider the sign to be positive

    h = [0 if i<0 else 1 for i in np.squeeze(sign_of_dot_product)]

    # remove extra un-used dimensions (convert this from a 2D to a 1D array)
#     h = np.flatten(h)

    # initialize the hash value to 0
    hash_value = 0

    n_planes = planes.shape[1]
    for i in range(n_planes):
        # increment the hash value by 2^i * h_i
        hash_value += np.power(2,i)*h[i]

    # cast hash_value as an integer
    hash_value = int(hash_value)

    return hash_value

In [ ]:
idx = 0
planes = planes_l[idx]  # get one 'universe' of planes to test the function
vec = normalize(np.random.normal(size=(1, N_DIMS)), axis=1, norm='l2')
print(f" The hash value for this vector,",
      f"and the set of planes at index {idx},",
      f"is {hash_value_of_vector(vec, planes)}")

## Creating a hash table

Given that you have a unique number for each vector (or tweet), You now want to create a hash table. You need a hash table, so that given a hash_id, you can quickly look up the corresponding vectors. This allows you to reduce your search by a significant amount of time.

We have given you the `make_hash_table` function, which maps the tweet vectors to a bucket and stores the vector there. It returns the `hash_table` and the `id_table`. The `id_table` allows you know which vector in a certain bucket corresponds to what tweet.

In [ ]:
def make_hash_table(vecs, planes):
    """
    Input:
        - vecs: list of vectors to be hashed.
        - planes: the matrix of planes in a single "universe", with shape (embedding dimensions, number of planes).
    Output:
        - hash_table: dictionary - keys are hashes, values are lists of vectors (hash buckets)
        - id_table: dictionary - keys are hashes, values are list of vectors id's
                            (it's used to know which tweet corresponds to the hashed vector)
    """

    # number of planes is the number of columns in the planes matrix
    num_of_planes = planes.shape[1]

    # number of buckets is 2^(number of planes)
    num_buckets = np.power(2, num_of_planes)
    
    # create the hash table as a dictionary.
    # Keys are integers (0,1,2.. number of buckets)
    # Values are empty lists
    hash_table = {key: [] for key in range(num_buckets)}

    # create the id table as a dictionary.
    # Keys are integers (0,1,2... number of buckets)
    # Values are empty lists
    id_table = {key: [] for key in range(num_buckets)}

    # for each vector in 'vecs'
    for i, v in enumerate(vecs):
        # calculate the hash value for the vector
        h = hash_value_of_vector(v, planes)

        # store the vector into hash_table at key h,
        # by appending the vector v to the list at key h
        hash_table[h].append(v)

        # store the vector's index 'i' (each document is given a unique integer 0,1,2...)
        # the key is the h, and the 'i' is appended to the list at key h
        id_table[h].append(i)

    return hash_table, id_table

In [ ]:
planes = planes_l[0]  # get one 'universe' of planes to test the function
vec = normalize(np.random.normal(size=(1, N_DIMS)), axis=1, norm='l2')
tmp_hash_table, tmp_id_table = make_hash_table(train_data, planes)

print(f"The hash table at key 0 has {len(tmp_hash_table[0])} document vectors")
print(f"The id table at key 0 has {len(tmp_id_table[0])}")
print(f"The first 5 document indices stored at key 0 of are {tmp_id_table[0][0:5]}")

In [ ]:
# Creating the hashtables
hash_tables = []
id_tables = []
for universe_id in range(N_UNIVERSES):  # there are 25 hashes
    print('working on hash universe #:', universe_id)
    planes = planes_l[universe_id]
    hash_table, id_table = make_hash_table(train_data, planes)
    hash_tables.append(hash_table)
    id_tables.append(id_table)

## Approximate K-NN

Implement approximate K nearest neighbors using locality sensitive hashing,
to search for documents that are similar to a given document at the
index `doc_id`.

###### Inputs
* `doc_id` is the index into the document list `all_tweets`.
* `v` is the document vector for the tweet in `all_tweets` at index `doc_id`.
* `planes_l` is the list of planes (the global variable created earlier).
* `k` is the number of nearest neighbors to search for.
* `num_universes_to_use`: to save time, we can use fewer than the total
number of available universes.  By default, it's set to `N_UNIVERSES`,
which is $25$ for this assignment.

The `approximate_knn` function finds a subset of candidate vectors that
are in the same "hash bucket" as the input vector 'v'.  Then it performs
the usual k-nearest neighbors search on this subset (instead of searching
through all 10,000 tweets).

In [ ]:
def cosine_similarity(u, v):
    return np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))

def nearest_neighbor(v, candidates, k=1):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    similarity_l = []

    # for each candidate vector...
    for row in candidates:
        # get the cosine similarity
        cos_similarity = cosine_similarity(v.flatten(), row)

        # append the similarity to the list
        similarity_l.append(cos_similarity)
        
    # sort the similarity list and get the indices of the sorted list
    sorted_ids = np.argsort(similarity_l)[::-1]
    similarity_l_sorted = np.sort(similarity_l)[::-1]
    
    # get the indices of the k most similar candidate vectors
    k_idx = sorted_ids[:k]
    similarities_k = similarity_l_sorted[:k]

    return k_idx, similarities_k


In [ ]:
def approximate_knn(id_tables, hash_tables, v, planes_l, k=1, num_universes_to_use=N_UNIVERSES):
    """Search for k-NN using hashes."""
    assert num_universes_to_use <= N_UNIVERSES

    # Vectors that will be checked as possible nearest neighbor
    vecs_to_consider_l = list()

    # list of document IDs
    ids_to_consider_l = list()

    # create a set for ids to consider, for faster checking if a document ID already exists in the set
    ids_to_consider_set = set()

    # loop through the universes of planes
    for universe_id in range(num_universes_to_use):

        # get the set of planes from the planes_l list, for this particular universe_id
        planes = planes_l[universe_id]

        # get the hash value of the vector for this set of planes
        hash_value = hash_value_of_vector(v, planes)

        # get the hash table for this particular universe_id
        hash_table = hash_tables[universe_id]

        # get the list of document vectors for this hash table, where the key is the hash_value
        document_vectors_l = hash_table[hash_value]

        # get the id_table for this particular universe_id
        id_table = id_tables[universe_id]

        # get the subset of documents to consider as nearest neighbors from this id_table dictionary
        new_ids_to_consider = id_table[hash_value]

#         # IMPLEMENT THIS!
#         if v in vocab:
#             return v

        # remove the id of the document that we're searching
#         if doc_id in new_ids_to_consider:
#             new_ids_to_consider.remove(doc_id)
#             print(f"removed doc_id {doc_id} of input vector from new_ids_to_search")

        # loop through the subset of document vectors to consider
        for i, new_id in enumerate(new_ids_to_consider):

            # if the document ID is not yet in the set ids_to_consider...
            if new_id not in ids_to_consider_set:
                # access document_vectors_l list at index i to get the embedding
                # then append it to the list of vectors to consider as possible nearest neighbors
                embedding = document_vectors_l[i]
                document_vector_at_i = vecs_to_consider_l.append(embedding)

                # append the new_id (the index for the document) to the list of ids to consider
                ids_to_consider_l.append(new_id)

                # also add the new_id to the set of ids to consider
                # (use this to check if new_id is not already in the IDs to consider)
                ids_to_consider_set.add(new_id)

    # Now run k-NN on the smaller set of vecs-to-consider.
#     print("Fast considering %d vecs" % len(vecs_to_consider_l))

    # convert the vecs to consider set to a list, then to a numpy array
    vecs_to_consider_arr = np.array(vecs_to_consider_l)

    # call nearest neighbors on the reduced list of candidate vectors
    nearest_neighbor_idx_l, similarities = nearest_neighbor(v, vecs_to_consider_arr, k=k)

    # Use the nearest neighbor index list as indices into the ids to consider
    # create a list of nearest neighbors by the document ids
    nearest_neighbor_ids = [ids_to_consider_l[idx]
                            for idx in nearest_neighbor_idx_l]

    return nearest_neighbor_ids, similarities


In [ ]:
# Word from search
trials = 200
for trial in range(trials):
    vec_to_search = normalize(np.random.normal(size=(1, N_DIMS)), axis=1, norm='l2')
    nearest_neighbor_ids, similarities = approximate_knn(
    id_tables, hash_tables, vec_to_search, planes_l, k=2, num_universes_to_use=2)
    if trial % 50 == 0:
#         print(vec_to_search)
#         print(train_data[nearest_neighbor_ids])
#         print(nearest_neighbor_ids)
#         print(similarities)
        pass

In [ ]:
nearest_neighbor_ids = approximate_knn(
    id_tables, hash_tables, vec_to_search, planes_l, k=5, num_universes_to_use=1)

In [ ]:
for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {train_data[neighbor_id]}")

In [ ]:
## Create planes and store them as pickle file
def create_hash_planes(n_dims, n_planes, n_universes, seed=None):
    # Set seed for reproducibility
    if seed:
        np.random.seed(seed)
    # Create a list with n_universes sets of n_planes of n_dims dimensions
    hashing_planes = [np.random.normal(size=(N_DIMS, N_PLANES)) for _ in range(N_UNIVERSES)]
    
    return hashing_planes

def get_all_embeddings():
    # Replace this function with get embeddings from database
    embeddings = train_data = normalize(np.random.rand(N_VECS, N_DIMS), axis=1, norm='l2')
    
    return embeddings

def make_hash_table(vecs, planes):
    """
    Input:
        - vecs: list of vectors to be hashed.
        - planes: the matrix of planes in a single "universe", with shape (embedding dimensions, number of planes).
    Output:
        - hash_table: dictionary - keys are hashes, values are lists of vectors (hash buckets)
        - id_table: dictionary - keys are hashes, values are list of vectors id's
                            (it's used to know which tweet corresponds to the hashed vector)
    """

    # number of planes is the number of columns in the planes matrix
    num_of_planes = planes.shape[1]

    # number of buckets is 2^(number of planes)
    num_buckets = np.power(2, num_of_planes)
    
    # create the hash table as a dictionary.
    # Keys are integers (0,1,2.. number of buckets)
    # Values are empty lists
    hash_table = {key: [] for key in range(num_buckets)}

    # create the id table as a dictionary.
    # Keys are integers (0,1,2... number of buckets)
    # Values are empty lists
    id_table = {key: [] for key in range(num_buckets)}

    # for each vector in 'vecs'
    for i, v in enumerate(vecs):
        # calculate the hash value for the vector
        h = hash_value_of_vector(v, planes)

        # store the vector into hash_table at key h,
        # by appending the vector v to the list at key h
        hash_table[h].append(v)

        # store the vector's index 'i' (each document is given a unique integer 0,1,2...)
        # the key is the h, and the 'i' is appended to the list at key h
        id_table[h].append(i)

    return hash_table, id_table

def create_hash_tables(hash_planes, embeddings, n_universes):
    # hash_table --> key (): value ()
    # id_table --> key(): value()
    hash_tables = []
    id_tables = []
    for universe_id in range(n_universes):  # there are 25 hashes
        print('working on hash universe #:', universe_id)
        universe_planes = hash_planes[universe_id]
        hash_table, id_table = make_hash_table(train_data, universe_planes)
        hash_tables.append(hash_table)
        id_tables.append(id_table)
    return hash_tables, id_tables

def create_pandas_universe_hash_table(N_UNIVERSES, id_tables, embeddings):
    pd_id_tables = pd.DataFrame()
    for universe in range(N_UNIVERSES):
        # Get id table for a certain universe
        universe_hash_table = id_tables[universe]
        # pandas id_tables
        pd_id_tables_universe = pd.DataFrame()
        for key in universe_hash_table.keys():
            # Get hash_ids to identify our vectors (skills)
            hash_ids = universe_hash_table[key]
            # Get embeddings belonging to this bucket in this universe
            universe_key_embeddings = pd.DataFrame(embeddings[hash_ids])
            # Create pandas dataframe
            pd_id_tables_bucket = pd.DataFrame(data=hash_ids, columns=['hash_id'])
            # Assign bucket
            pd_id_tables_bucket['hash_bucket'] = key
            # Convert to int8
            pd_id_tables_bucket['hash_bucket'] = pd.to_numeric(pd_id_tables_bucket['hash_bucket'], downcast='integer')
            pd_id_tables_bucket = pd.concat([pd_id_tables_bucket, universe_key_embeddings], axis=1)
            # COnvert to float16
            for col in pd_id_tables_bucket:
                if pd_id_tables_bucket[col].dtype == np.float64:
                    pd_id_tables_bucket[col] = pd_id_tables_bucket[col].astype(np.float16)
            if not pd_id_tables_bucket.empty:
                pd_id_tables_universe = pd.concat([pd_id_tables_universe, pd_id_tables_bucket], axis=0)
        pd_id_tables_universe['universe'] = universe
        pd_id_tables = pd.concat([pd_id_tables, pd_id_tables_universe], axis=0)
    return pd_id_tables

In [ ]:
seed = 2
# Create planes
hash_planes = create_hash_planes(N_DIMS, N_PLANES, N_UNIVERSES)

# Get embeddings (skill vectors from the ddbb)
embeddings = get_all_embeddings()

# Create hashing tables
## - hash_tables is a list of n_universes lists with n_buckets dictionaries where
## key = 'bucket_id' and value = embedding belongs to this bucket in this universe.
## - id_tables is a list of n_universes with n_buckets dictionaries where 
## key = 'bucket_id' and value = embedding_id that belongs to this bucket in this universe.
hash_tables, id_tables = create_hash_tables(hash_planes, embeddings, N_UNIVERSES)

pd_id_tables = create_pandas_universe_hash_table(N_UNIVERSES, id_tables, embeddings)

In [ ]:
pd_id_tables.memory_usage(index=True).sum()/1024/1024

In [ ]:
pd_id_tables.dtypes